# Project Title
### Data Engineering Capstone Project

#### Project Summary

### Camilo Montenegro


The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import configparser
from datetime import datetime
import os
import pandas as pd
from pyspark.sql.types import MapType, StringType, StructType, IntegerType, DoubleType, TimestampType, StructField
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, split
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

### Step 1: Scope the Project and Gather Data

#### Scope 

This capstone project is intended to cover the topics learned during this Nanodegree, and execute some final analysis over the data set so an analytics dashboard can be created in a further iteration. 

The results obtained from this data discovery can be useful to understand how the Tennis world has been moving during the years. So this data set analysis can be used by various actors such as national and world tennis federations, trainers, tennis players, and fans. They can also discover how was the performance of a player in some or specific tournaments, how that player had been developing his/her skills based on match results, and lots more of analysis. This project easily can be extended to a web app dashboard in which fans can execute user-friendly queries and discover new information.

Here we made use of some tools such as Spark and Pandas in a Python > 3.x environment

The project's plan is defined in the next steps:
1. Gather the required information, in this cause we're using an ATP Tennis Data Set that will be explained in the upcoming section
2. Analyze the documentation of the data to understand which analysis can be performed using the existing data
3. Extract the information into the current spark session, so the following data processing steps can be execute
4. Perform data cleaning removing unwanted data such as null, empty and duplicates tuples
5. Create the data pipeline in which spark is in charge of creating the tables based on the previous information
6. Run some quality checks after the pipeline is performed to be sure that the data is consistent
7. Perform some analytics query to show the potential of using spark and the created data model

#### Describe and Gather Data 

- The data set used in this project was extracted and loaded from this Github repository: https://github.com/JeffSackmann/tennis_atp.

- This contains master ATP player file, historical rankings, results, and match stats.

- The player file columns are player_id, first_name, last_name, hand, birth_date, country_code.

- The columns for the ranking files are ranking_date, ranking, player_id, ranking_points (where available).

- ATP rankings are mostly complete from 1985 to the present. 1982 is missing, and rankings from 1973-1984 are only intermittent.

- Results and stats: There are up to three files per season: One for tour-level main draw matches (e.g. 'atp_matches_2014.csv'), one for tour-level qualifying and challenger main-draw matches, and one for futures matches.

- Most of the columns in the results files are self-explanatory. There's also included a matches_data_dictionary.txt file to spell things out a bit more.


In [2]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df = spark.read.load("atpTennisData/tennis_atp/atp_players.csv",
                     format="csv", sep=",", inferSchema="true").toDF(
  'playerId','firstName','lastName','hand','bornDate','country'
)


In [134]:
df.printSchema()

root
 |-- ranking_date: string (nullable = true)
 |-- rank: string (nullable = true)
 |-- player: string (nullable = true)
 |-- points: string (nullable = true)



In [4]:
df.count()

54938

In [5]:
df.show(5, truncate=True)

+--------+--------------+--------+----+--------+-------+
|playerId|     firstName|lastName|hand|bornDate|country|
+--------+--------------+--------+----+--------+-------+
|  100001|       Gardnar|  Mulloy|   R|19131122|    USA|
|  100002|        Pancho|  Segura|   R|19210620|    ECU|
|  100003|         Frank| Sedgman|   R|19271002|    AUS|
|  100004|      Giuseppe|   Merlo|   R|19271011|    ITA|
|  100005|Richard Pancho|Gonzales|   R|19280509|    USA|
+--------+--------------+--------+----+--------+-------+
only showing top 5 rows



In [ ]:
#write to parquet
#df.write.partitionBy('country','playerId').format('parquet').save('atpTennisData/spark/atp_playersV3.parquet')

In [7]:
df=spark.read.parquet("atpTennisData/spark/atp_playersV3.parquet/")

### Step 2: Explore and Assess the Data
#### Explore & Clean the Data 
After reading all the information using spark environment, it was necesary to understand the schema that we were working on. So for all the gather information the data schema was discovered. Afterwards, we perfomed the spark describe function to identify the behaviour of the data (count, mean, stddev, min, max) and missing values. With this in mind, and for each table we dropped the tuples that had one or more missing values.  
 

In [6]:
# Performing cleaning tasks here
df.printSchema()

root
 |-- playerId: integer (nullable = true)
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- hand: string (nullable = true)
 |-- bornDate: integer (nullable = true)
 |-- country: string (nullable = true)



In [8]:
df.count()

54938

In [10]:
df.show(5,truncate=True)

+--------+--------------+--------+----+--------+-------+
|playerId|     firstName|lastName|hand|bornDate|country|
+--------+--------------+--------+----+--------+-------+
|  100001|       Gardnar|  Mulloy|   R|19131122|    USA|
|  100002|        Pancho|  Segura|   R|19210620|    ECU|
|  100003|         Frank| Sedgman|   R|19271002|    AUS|
|  100004|      Giuseppe|   Merlo|   R|19271011|    ITA|
|  100005|Richard Pancho|Gonzales|   R|19280509|    USA|
+--------+--------------+--------+----+--------+-------+
only showing top 5 rows



In [11]:
df.describe().show()

+-------+-----------------+--------------+--------+-----+--------------------+-------+
|summary|         playerId|     firstName|lastName| hand|            bornDate|country|
+-------+-----------------+--------------+--------+-----+--------------------+-------+
|  count|            54938|         54761|   54896|49152|               43356|  54883|
|   mean|137411.8036331865|          null|     1.0| null|1.9816213222206846E7|   null|
| stddev|33843.15964803341|          null|     NaN| null|   321641.3326366638|   null|
|    min|           100001|"Jason ""Jj"""|       1|    A|              185903|    AFG|
|    max|           209903|       Zygmunt|  Zysset|    U|            20050210|    ZIM|
+-------+-----------------+--------------+--------+-----+--------------------+-------+



In [13]:
df_spark_valid = df.dropna(how = "any", subset = ["firstName", "lastName","hand","bornDate","country"])

In [15]:
df_spark_valid.count()

42055

In [44]:
df_spark_valid.printSchema()

root
 |-- playerId: integer (nullable = true)
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- hand: string (nullable = true)
 |-- bornDate: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- ts_bornDate: timestamp (nullable = true)



In [43]:
df_spark_valid.show(5,truncate=True)

+--------+--------------+--------+----+--------+-------+-------------------+
|playerId|     firstName|lastName|hand|bornDate|country|        ts_bornDate|
+--------+--------------+--------+----+--------+-------+-------------------+
|  100001|       Gardnar|  Mulloy|   R|19131122|    USA|1913-11-22 00:00:00|
|  100002|        Pancho|  Segura|   R|19210620|    ECU|1921-06-20 00:00:00|
|  100003|         Frank| Sedgman|   R|19271002|    AUS|1927-10-02 00:00:00|
|  100004|      Giuseppe|   Merlo|   R|19271011|    ITA|1927-10-11 00:00:00|
|  100005|Richard Pancho|Gonzales|   R|19280509|    USA|1928-05-09 00:00:00|
+--------+--------------+--------+----+--------+-------+-------------------+
only showing top 5 rows



In [50]:
	
df_spark_ranks = spark.read.load("atpTennisData/tennis_atp/atp_rankings_*.csv",
                     format="csv", sep=",", inferSchema="true",header=True)

In [51]:
df_spark_ranks.printSchema()

root
 |-- ranking_date: integer (nullable = true)
 |-- rank: integer (nullable = true)
 |-- player: integer (nullable = true)
 |-- points: integer (nullable = true)



In [52]:
df_spark_ranks.count()

2857079

In [53]:
df_spark_ranks.show(5,truncate=True)

+------------+----+------+------+
|ranking_date|rank|player|points|
+------------+----+------+------+
|    20000110|   1|101736|  4135|
|    20000110|   2|102338|  2915|
|    20000110|   3|101948|  2419|
|    20000110|   4|103017|  2184|
|    20000110|   5|102856|  2169|
+------------+----+------+------+
only showing top 5 rows



In [54]:
df_spark_ranks.describe().show()

+-------+--------------------+------------------+------------------+------------------+
|summary|        ranking_date|              rank|            player|            points|
+-------+--------------------+------------------+------------------+------------------+
|  count|             2857079|           2857079|           2857079|           2509664|
|   mean|2.0036640241374496E7| 827.7547946696609|109680.05734108157|114.90980426065003|
| stddev|    97916.8959197867|506.91610232538534|17974.363400013513| 416.5161568367033|
|    min|            19730827|                 1|            100001|                 1|
|    max|            20200309|              2271|            209866|             16950|
+-------+--------------------+------------------+------------------+------------------+



In [60]:
df_spark_valid_ranks = df_spark_ranks.dropna(how = "any", subset = ["points"])

In [61]:
df_spark_valid_ranks.describe().show()

+-------+-------------------+-----------------+------------------+------------------+
|summary|       ranking_date|             rank|            player|            points|
+-------+-------------------+-----------------+------------------+------------------+
|  count|            2509664|          2509664|           2509664|           2509664|
|   mean|2.005838226890014E7|866.7871468053094|110556.60943138205|114.90980426065003|
| stddev|  82402.91547965597|515.6314137517035|18872.154068893873| 416.5161568367033|
|    min|           19900101|                1|            100019|                 1|
|    max|           20200309|             2271|            209866|             16950|
+-------+-------------------+-----------------+------------------+------------------+



In [62]:
df_spark_valid_ranks = df_spark_valid_ranks.withColumn("ts_rankingDate", toTimeStampFormat("ranking_date"))

In [63]:
df_spark_valid_ranks.printSchema()

root
 |-- ranking_date: integer (nullable = true)
 |-- rank: integer (nullable = true)
 |-- player: integer (nullable = true)
 |-- points: integer (nullable = true)
 |-- ts_rankingDate: timestamp (nullable = true)



In [ ]:
df_spark_valid_ranks.show(5,truncate=True)

In [28]:
df_spark_matches = spark.read.load("atpTennisData/tennis_atp/atp_matches_*.csv",
                     format="csv", sep=",", inferSchema="true",header=True)

In [29]:
df_spark_matches.printSchema()

root
 |-- tourney_id: string (nullable = true)
 |-- tourney_name: string (nullable = true)
 |-- surface: string (nullable = true)
 |-- draw_size: string (nullable = true)
 |-- tourney_level: string (nullable = true)
 |-- tourney_date: string (nullable = true)
 |-- match_num: string (nullable = true)
 |-- winner_id: string (nullable = true)
 |-- winner_seed: string (nullable = true)
 |-- winner_entry: string (nullable = true)
 |-- winner_name: string (nullable = true)
 |-- winner_hand: string (nullable = true)
 |-- winner_ht: string (nullable = true)
 |-- winner_ioc: string (nullable = true)
 |-- winner_age: string (nullable = true)
 |-- loser_id: string (nullable = true)
 |-- loser_seed: string (nullable = true)
 |-- loser_entry: string (nullable = true)
 |-- loser_name: string (nullable = true)
 |-- loser_hand: string (nullable = true)
 |-- loser_ht: string (nullable = true)
 |-- loser_ioc: string (nullable = true)
 |-- loser_age: string (nullable = true)
 |-- score: string (nullable 

In [30]:
columns_to_drop = ['winner_seed','winner_entry','winner_ioc','loser_seed','loser_entry','w_ace','l_ace','w_svpt','l_svpt','w_1stIn','l_1stIn','w_SvGms','lSvGms','w_bpSaved','w_bpFaced','l_bpSaved','l_bpFaced']
df_spark_matches = df_spark_matches.drop(*columns_to_drop)

In [33]:
df_spark_matches.printSchema()

root
 |-- tourney_id: string (nullable = true)
 |-- tourney_name: string (nullable = true)
 |-- surface: string (nullable = true)
 |-- draw_size: string (nullable = true)
 |-- tourney_level: string (nullable = true)
 |-- tourney_date: string (nullable = true)
 |-- match_num: string (nullable = true)
 |-- winner_id: string (nullable = true)
 |-- winner_name: string (nullable = true)
 |-- winner_hand: string (nullable = true)
 |-- winner_ht: string (nullable = true)
 |-- winner_age: string (nullable = true)
 |-- loser_id: string (nullable = true)
 |-- loser_name: string (nullable = true)
 |-- loser_hand: string (nullable = true)
 |-- loser_ht: string (nullable = true)
 |-- loser_ioc: string (nullable = true)
 |-- loser_age: string (nullable = true)
 |-- score: string (nullable = true)
 |-- best_of: string (nullable = true)
 |-- round: string (nullable = true)
 |-- minutes: string (nullable = true)
 |-- w_df: string (nullable = true)
 |-- w_1stWon: string (nullable = true)
 |-- w_2ndWon: 

In [32]:
df_spark_matches.count()

814534

In [31]:
df_spark_matches.describe().show()

+-------+----------+----------------+-------+------------------+------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------+------------------+---------+-----------------+------------+------------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+-----------------+------------------+
|summary|tourney_id|    tourney_name|surface|         draw_size|     tourney_level|        tourney_date|         match_num|         winner_id|       winner_name|       winner_hand|         winner_ht|        winner_age|          loser_id|        loser_name|loser_hand|          loser_ht|loser_ioc|        loser_age|       score|           best_of|             round|          minutes|             w_df|          

In [34]:
df_spark_matches.show(5, truncate = True)

+--------------------+------------+-------+---------+-------------+------------+---------+---------+-----------------+-----------+---------+-------------+--------+------------------+----------+--------+---------+-------------+----------+-------+-----+-------+----+--------+--------+----+--------+--------+-------+-----------+------------------+----------+-----------------+
|          tourney_id|tourney_name|surface|draw_size|tourney_level|tourney_date|match_num|winner_id|      winner_name|winner_hand|winner_ht|   winner_age|loser_id|        loser_name|loser_hand|loser_ht|loser_ioc|    loser_age|     score|best_of|round|minutes|w_df|w_1stWon|w_2ndWon|l_df|l_1stWon|l_2ndWon|l_SvGms|winner_rank|winner_rank_points|loser_rank|loser_rank_points|
+--------------------+------------+-------+---------+-------------+------------+---------+---------+-----------------+-----------+---------+-------------+--------+------------------+----------+--------+---------+-------------+----------+-------+-----+-

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model

![Hola](./images/ConceptualDataModel.png)

As you can see, the proposed Conceptual Data Model is similar to a Star Schema in the sense that the atp_matches_table represent our fact table and the other tables are the dimension tables that contains extra information needed to perform some queries. This model tries to keep a normalize and denormalize ratio in the sense that new queries can appear after this model decision have been take. Here you can see as well, that the lines shows the foreign keys relations that this schema keeps.

#### 3.2 Mapping Out Data Pipelines
After we'd gather all the information from our raw databases it's necessary to perform the ETL -> transformation and load phase processes. 
1. In the last steps, we Extract the information from the databases
2. Then, we understood the data behavior so we can make design decisions to make the respective Transformations
3. Afterward, we remove some duplicate and missing data and select only those useful columns
4. We needed to create a time table for our data model star schema, for this, each possible table had to have a timestamp format column depending on each value row. That's why the 'user-defined function (UDF) was created before loading the data.
5. For each possible date, we create that timestamp column
6. Next, we load a temp view table using CreateOrReplaceTempView spark function. This performed for each dimension table and the fact table as well.
7. Then in the load step, we create the time table based on the timestamps in the fact table and some udf functions to obtain data such as day, week, month, year, and so on so forth.
8. Lastly, we performed the respective quality checks that are well explained in its section

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [38]:
toTimeStampFormat = udf(lambda x: pd.to_datetime(x,format="%Y%m%d"),TimestampType())

### Step 1: Select only the useful columns from our fact table (atp_matches) and save it to a spark temp table

In [ ]:
df_spark_matches_valid = df_spark_matches.withColumn("ts_tourneyDate", toTimeStampFormat("tourney_date"))

In [69]:
df_spark_matches_valid.createOrReplaceTempView('atp_matches_table')

### Step 2: Select only the useful columns from our dimension table (atp_players) and save it to a spark temp table

In [42]:
df_spark_valid = df_spark_valid.withColumn("ts_bornDate", toTimeStampFormat("bornDate"))

In [70]:
df_spark_valid.createOrReplaceTempView('atp_players_table')

### Step 3: Select only the useful columns from our dimension table (atp_ranks) and save it to a spark table

In [62]:
df_spark_valid_ranks = df_spark_valid_ranks.withColumn("ts_rankingDate", toTimeStampFormat("ranking_date"))

In [71]:
df_spark_valid_ranks.createOrReplaceTempView('atp_ranks_tables')

### Step 4: Create time mandatory dimension (time_table) and save it to a spark temp table

In [117]:
# create datetime column from original timestamp column
spark.udf.register("get_hour", lambda x: int(x.hour))
spark.udf.register("get_day", lambda x: int(x.day))
spark.udf.register("get_week", lambda x: int(x.isocalendar()[1]))
spark.udf.register("get_month", lambda x: int(x.month))
spark.udf.register("get_year", lambda x: int(x.year))
spark.udf.register("get_dayofweek", lambda x: int(x.weekday()))

# extract columns to create time table
time_table = spark.sql("""
SELECT distinct ts_tourneyDate as ts, get_day(ts_tourneyDate) as day, get_week(ts_tourneyDate) as week, get_month(ts_tourneyDate) as month, get_year(ts_tourneyDate) as year, get_dayofweek(ts_tourneyDate) as dayofweek
from atp_matches_table
""")

In [118]:
#this can take some time
time_table.show(5, truncate=True)

+-------------------+---+----+-----+----+---------+
|                 ts|day|week|month|year|dayofweek|
+-------------------+---+----+-----+----+---------+
|2016-05-16 00:00:00| 16|  20|    5|2016|        0|
|2012-02-27 00:00:00| 27|   9|    2|2012|        0|
|2011-09-12 00:00:00| 12|  37|    9|2011|        0|
|2008-12-01 00:00:00|  1|  49|   12|2008|        0|
|2019-09-09 00:00:00|  9|  37|    9|2019|        0|
+-------------------+---+----+-----+----+---------+
only showing top 5 rows



In [119]:
time_table.createOrReplaceTempView('time_table')

#### 4.2 Data Quality Checks
- The quality checks performed include source/count completeness in each table, integrity constraints on the relational database such as datatypes verification and primary key and foreign key consistency, this was checked performing some SQL queries

- Next you can find the results


- Verify source count check to ensure completeness

In [105]:
def checkCountItems(spark,table,quantity):
    sql_table = spark.sql(f"""
    SELECT count(*) from {table}
    """)
    amount = sql_table.first()[0]
    if(amount<quantity):
         raise ValueError(f"Data quality check failed. {table} returned no results")
    else:
        return f"Data quality check sucess. {table} returned {amount} results"

    Table atp_matches_table

In [107]:
checkCountItems(spark,'atp_matches_table',1)

'Data quality check sucess. atp_matches_table returned 814534 results'

    Table atp_players_table

In [108]:
checkCountItems(spark,'atp_players_table',1)

'Data quality check sucess. atp_players_table returned 42055 results'

    Table atp_ranks_tables

In [109]:
checkCountItems(spark,'atp_ranks_tables',1)

'Data quality check sucess. atp_ranks_tables returned 2509664 results'

    Table time_table

In [141]:
checkCountItems(spark,'time_table',1)

'Data quality check sucess. time_table returned 4063 results'

- Check datatypes of the tables

In [112]:
def count_column_types(spark_df):
    """Count number of columns per type"""
    return pd.DataFrame(spark_df.dtypes).groupby(1, as_index=False)[0].agg({'count':'count', 'names': lambda x: " | ".join(set(x))}).rename(columns={1:"type"})

In [113]:
count_column_types(df_spark_matches_valid)

,type,count,names
0,string,33,round | best_of | tourney_date | w_1stWon | ma...
1,timestamp,1,ts_tourneyDate


In [114]:
count_column_types(df_spark_valid)

,type,count,names
0,int,2,bornDate | playerId
1,string,4,firstName | hand | lastName | country
2,timestamp,1,ts_bornDate


In [115]:
count_column_types(df_spark_valid_ranks)

,type,count,names
0,int,4,rank | ranking_date | points | player
1,timestamp,1,ts_rankingDate


In [120]:
count_column_types(time_table)

,type,count,names
0,string,5,year | day | month | week | dayofweek
1,timestamp,1,ts


- Perfom some join queries to check primary and foreign keys integrity

      Get the Tourney_ID, Tourney_date, Tourney_name, level, score, name of the player, hand and origin country of all those players that its origin country is Colombia (COL) order desc by player's last name

In [131]:
query = spark.sql("""
Select ATPm.tourney_id, ATPm.ts_tourneyDate, ATPm.tourney_name, ATPm.tourney_level, ATPm.score, ATPp.firstName, ATPp.lastName, ATPp.hand, ATPp.country
from atp_matches_table ATPm
join atp_players_table ATPp
on ATPm.winner_id=ATPp.playerId
where(ATPp.country="COL") order by ATPp.lastName DESC
""")

In [132]:
query.show(20, truncate=True)

+--------------------+-------------------+------------+-------------+---------------+---------+---------------+----+-------+
|          tourney_id|     ts_tourneyDate|tourney_name|tourney_level|          score|firstName|       lastName|hand|country|
+--------------------+-------------------+------------+-------------+---------------+---------+---------------+----+-------+
|2012-M-FU-COL-04A...|2012-10-01 00:00:00| Colombia F4|            S|        6-1 6-0| Barlaham|Zuluaga Gaviria|   R|    COL|
|2016-M-FU-COL-08A...|2016-11-21 00:00:00| Colombia F8|            S|    1-6 6-2 6-2| Barlaham|Zuluaga Gaviria|   R|    COL|
|2013-M-FU-COL-04A...|2013-08-12 00:00:00| Colombia F4|            S|    1-6 6-4 6-3| Barlaham|Zuluaga Gaviria|   R|    COL|
|2014-M-FU-COL-06A...|2014-09-15 00:00:00| Colombia F6|            S|     7-6(5) 6-4|  Steffen|        Zornosa|   L|    COL|
|2015-M-FU-COL-09A...|2015-11-02 00:00:00| Colombia F9|            S|4-6 6-3 5-2 RET|  Steffen|        Zornosa|   L|    COL|


In [133]:
query.count()

6142

      Get the Tourney_ID, Tourney_date, Tourney_name, level, score, name of the player, hand, origin country, ranking and ranking date of all those players that its origin country is Colombia (COL) order desc by player's last name
      
       Here we have to take into account that the join performed is neither right or left join so that's why we got more tuples, the rank and ranking_date is repeated in all the possible records. This was just and example to show how we can use the ranking table as well

In [137]:
query2 = spark.sql("""
Select ATPm.tourney_id, ATPm.ts_tourneyDate, ATPm.tourney_name, ATPm.tourney_level, ATPm.score, ATPp.firstName, ATPp.lastName, ATPp.hand, ATPp.country, ATPr.rank, ATPr.ranking_date
from atp_matches_table ATPm
join atp_players_table ATPp
on ATPm.winner_id=ATPp.playerId
join atp_ranks_tables ATPr
on ATPp.playerId=ATPr.player
where(ATPp.country="COL") order by ATPp.lastName DESC
""")

In [138]:
query2.show(20, truncate=True)

+--------------------+-------------------+------------+-------------+-----------+---------+---------------+----+-------+----+------------+
|          tourney_id|     ts_tourneyDate|tourney_name|tourney_level|      score|firstName|       lastName|hand|country|rank|ranking_date|
+--------------------+-------------------+------------+-------------+-----------+---------+---------------+----+-------+----+------------+
|2013-M-FU-COL-04A...|2013-08-12 00:00:00| Colombia F4|            S|1-6 6-4 6-3| Barlaham|Zuluaga Gaviria|   R|    COL|1668|    20130225|
|2013-M-FU-COL-04A...|2013-08-12 00:00:00| Colombia F4|            S|1-6 6-4 6-3| Barlaham|Zuluaga Gaviria|   R|    COL|1683|    20121224|
|2013-M-FU-COL-04A...|2013-08-12 00:00:00| Colombia F4|            S|1-6 6-4 6-3| Barlaham|Zuluaga Gaviria|   R|    COL|1672|    20130218|
|2013-M-FU-COL-04A...|2013-08-12 00:00:00| Colombia F4|            S|1-6 6-4 6-3| Barlaham|Zuluaga Gaviria|   R|    COL|1655|    20121119|
|2013-M-FU-COL-04A...|2013-

In [140]:
query2.count()

3305167

#### 4.3 Data dictionary 

- Is possible to find the data dictionary in the next local file path: atpTennisData/tennis_atp/matches_data_dictionary.txt. Or in this Github file: https://github.com/JeffSackmann/tennis_atp/blob/master/matches_data_dictionary.txt


### Clearly state the rationale for the choice of tools and technologies for the project.
The tools and technologies used in this project were mainly Spark and Python, though the raw data and spark host can be achive using AWS services such as EMR, S3 buckets and Redshift.

So to sum up the reasons of choosing these techonologies are:
- Easily scale up using the previous mention techonology
- Rapidly query, analyze, and transform data at scale
- Can handle big amount of data, in this project we worked with:
    - 814.534 Matches
    - 42.055 Players
    - 2.509.664 Ranks
    - 4.063 Time data
    - In total: 3.370.316 data tuples with 6 attributes in average each tuple. So this's around 20.221.896 raw data loaded in a Apache Spark
- Great perfomance in simple and complex queries
- Is possible to handle partitions so the queries and processing can be even faster

### Propose how often the data should be updated and why.
- This dataset keeps the information files per year for the matches and per 10 years for the rankings, though each year new tournaments, matches, rankings, and players can be appended to the database. Usually, this kind of sports run by 6 months cycles A.K.A seasons that approx. have a length of 6 months. So, my suggestion could be to update the data each season iterative cuz the information of 2019-2nd season will be not updated by the 2020-1st season. So the pipeline to create, let's say in Airflow, will be performed every 6 months, can be performed each month if it's required, though.

### The data was increased by 100x.
- In this case, it will be necessary to evaluate mainly the architecture that Spark is running in. So the first step will be to increase the nodes in which the workers perform the different tasks, It's not possible to know exactly how much nodes will be needed for the 2B (2M*100x) data, with this in mind, it is necessary to perform some trial and error to fix the system architecture

### The data populates a dashboard that must be updated on a daily basis by 7am every day.
- Taking into account the data structure of the raw files in this project, it's necessary to append the new information in the current year file with the proper timestamp, so when we execute the Airflow pipeline with a @daily schedule, it only gathers the new information according to the previous day. With this in mind, those big created queries need to allocate more machine resources to perform better all the days before the business needs the dashboard. 

### The database needed to be accessed by 100+ people
- If the database is only used for reading purposes and in a non-update data window time it should be not many problems regarding ACID properties. Nevertheless, if more than 100+ people access just for reading purposes, the master node or driver needs to have more capacity of scheduling tasks to its works, as the driver increases so the workers.
- The more people accessing the database the more cpu resources you need to get a fast experience. By using a distributed database you can to improve your replications and partitioning to get faster query results for each user.